In [5]:
%cd /afs/crc.nd.edu/user/m/msaebi/Public/Code-SF-HON/
import pandas as pd
import time
import datetime
from datetime import datetime
from collections import defaultdict
import glob
import os
import itertools 


/afs/crc.nd.edu/user/m/msaebi/Public/Code-SF-HON


In [6]:
def process_df(year):
    #data=pd.read_csv('data/moves/moves_cleaned_'+year+'.txt', sep="|", header=None, error_bad_lines=False,low_memory=False)
    data=pd.read_csv('data/moves/moves_cleaned_'+year+'.txt', sep="|", header=None, engine='python',error_bad_lines=False)
    data=data.reset_index()
    new_header = data.iloc[0] #grab the first row for the header
    data = data[1:100] #take the data less the header row
    data.columns = new_header
    r_data=pd.DataFrame(data, columns = ["VESSEL ID","MOVE ID","PLACE ID","ARRIVAL DATE","SAIL DATE"])
    r_data['arrival']=r_data.apply(lambda row: get_date(row['ARRIVAL DATE']), axis=1)
    return data

def get_date(row):
    return datetime.strptime(row, '%Y-%m-%d %H:%M:%S').date()

def check_range(row,year1,lb1,ub1,year2,lb2,ub2):
    if year1 == year2:
        if (row.year == year1 and lb1<= row.month <= ub1):
            return True
    elif year1 != year2:
        if (row.year == year1 and lb1<= row.month <= ub1) or (row.year == year2 and lb2<= row.month <= ub2):
            return True
    
    return False


### Read the data

In [7]:
#from IPython.display import display, HTML
def create_trace_write(df,year):
    IDs=set(df['VESSEL ID'].tolist()) # all the unique vessel ids
    ID2 = sorted([int(i) for i in IDs]) 
    traces = defaultdict(list)
    print("start")
    for ids in ID2:
        v_df=df[df['VESSEL ID']==str(ids)] #create a df for each vessel
        for index, row in v_df.iterrows(): #iterate thought each vessel data frame and create the trace of moves 
            traces[str(ids)].append(int(row['PLACE ID']))
    traces_final= defaultdict(list)        
    for key, value in traces.items():
        traces_final[key]=[item for item, grp in itertools.groupby(value)]  
    
    #write it to the txt file   
    with open('data/'+year+'/traces_'+ year+'.txt', 'w') as f:
        for key, value in traces_final.items():
            f.write(str(key)+' '+','.join(map(repr,value ))+'\n')
    print('done')

In [8]:
df_2015 = process_df('2015')


In [13]:
create_trace_write(df_2015,'2015')

start


NameError: name 'itertools' is not defined

In [13]:
traces_final= defaultdict(list)
for key, value in traces.items():
        traces_final[key]=[item for item, grp in itertools.groupby(value)] 

In [ ]:
IDs=set(df['VESSEL ID'].tolist()) # all the unique vessel ids
ID2 = sorted([int(i) for i in IDs]) 
traces = defaultdict(list)
#ID2=['31']
print("start")
for ids in ID2:
    v_df=df[df['VESSEL ID']==str(ids)] #create a df for each vessel
    for index, row in v_df.iterrows(): #iterate thought each vessel data frame and create the trace of moves 
        traces[str(ids)].append(int(row['PLACE ID']))
    
    for key, value in traces.items():
        traces_final[key]=[item for item, grp in itertools.groupby(value)]  
#write it to the txt file   

with open('data/2012/'+ filename+'.txt', 'w') as f:
    for key, value in traces.items():
         f.write(str(key)+' '+','.join(map(repr,value ))+'\n')

### Removing duplicates

In [35]:
def open_trace_file(file):
    print(file)
    traces=defaultdict(list)
    lines=open(file,'r').readlines()
    for line in lines:
        l=line.split('\n')[0].split(' ')
        traces[l[0]]=[int(i) for i in l[1].split(',')]
    return traces

In [18]:
import glob
import os
import itertools 
year='1997'
files=glob.glob(os.path.expanduser('~/Public/Code-SF-HON/data/'+year+'/traces_'+year+'.txt'))

traces=open_trace_file(files)    
traces_final= defaultdict(list)
for key, value in traces.items():
    traces_final[key]=[item for item, grp in itertools.groupby(value)]   

with open(file.replace('.txt','-2.txt'), 'w') as f:
    for key, value in traces_final.items():
        f.write(str(key)+' '+','.join(map(repr,value ))+'\n')


/afs/crc.nd.edu/user/m/msaebi/Public/Code-SF-HON/data/1997/traces_1997.txt


In [48]:
#remove duplicated port in each vessel trace. Don't need to do if you use clean_moves
fil=os.path.expanduser('~/Public/Code-SF-HON/data/1997/traces_1997.txt')
all_traces=open_trace_file(fil)
all_vessels=list(all_traces.keys())
print(len(all_traces.keys()))

for year in ['1999','2002','2005','2008','2012','2015','2018']:
    file=os.path.expanduser('~/Public/Code-SF-HON/data/'+year+'/traces_'+year+'.txt')
    traces=open_trace_file(file)    
    i=0
    all_vessels.extend(list(traces.keys()))
    for key,val in traces.items():
        if key in all_traces.keys():
            i+=1
            all_traces[key].extend(val)
        else:
            all_traces[key]=val
    print(i,len(all_traces.keys()))

with open('data/traces_1997_2018.txt', 'w') as f:
    for key, value in all_traces.items():
        f.write(str(key)+' '+','.join(map(repr,value ))+'\n')

    


/afs/crc.nd.edu/user/m/msaebi/Public/Code-SF-HON/data/1997/traces_1997.txt
30557
/afs/crc.nd.edu/user/m/msaebi/Public/Code-SF-HON/data/1999/traces_1999.txt
26122 34806
/afs/crc.nd.edu/user/m/msaebi/Public/Code-SF-HON/data/2002/traces_2002.txt
25791 40918
/afs/crc.nd.edu/user/m/msaebi/Public/Code-SF-HON/data/2005/traces_2005.txt
27973 48000
/afs/crc.nd.edu/user/m/msaebi/Public/Code-SF-HON/data/2008/traces_2008.txt
31412 60286
/afs/crc.nd.edu/user/m/msaebi/Public/Code-SF-HON/data/2012/traces_2012.txt
31902 60288
/afs/crc.nd.edu/user/m/msaebi/Public/Code-SF-HON/data/2015/traces_2015.txt
31319 91947
/afs/crc.nd.edu/user/m/msaebi/Public/Code-SF-HON/data/2018/traces_2018.txt
55850 104626
